<a href="https://colab.research.google.com/github/ugur-sa/Bachelorarbeit/blob/main/SentimentTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hole HTML-Dokument der Nachricht über dessen Link und bereinige es grob

In [29]:
from bs4 import BeautifulSoup
import requests

res = requests.get("https://finance.yahoo.com/news/us-stocks-p-500-wobbles-210042198.html")

soup = BeautifulSoup(res.text, "html.parser")

for meta in soup.find_all("meta"):
    meta.decompose()

for script in soup.find_all("script"):
    script.decompose()

for noscript in soup.find_all("noscript"):
    noscript.decompose()

for link in soup.find_all("link"):
    link.decompose()

for style in soup.find_all("style"):
    style.decompose()

for html in soup.find_all('html'):
    html.unwrap()

# title = soup.find("title").text

soup = soup.find("div", attrs={'class':'caas-body'})

Hier wird weiter gearbeitet mit dem Div-Element, dass den Nachrichtentext enthält. Hierbei wird der pure Text extrahiert und Listen, Buttons und die Tags selber entfernt.

In [ ]:
for div in soup.find_all('div'):
    div.unwrap()

for ul in soup.find_all('ul'):
    # Finde das unmittelbar vorhergehende <p>-Tag, wenn vorhanden
    p_tag = ul.find_previous_sibling('p')

    # Wenn ein <p>-Tag gefunden wurde, entferne es
    if p_tag:
        p_tag.decompose()

# Entferne alle <div>-Elemente
for ul in soup.find_all("ul"):
    ul.decompose()

for button in soup.find_all("button"):
    button.decompose()

for p_tag in soup.find_all('p', string="©2023 Bloomberg L.P."):
    p_tag.decompose()

# text_lines = [p.get_text() + '\n' for p in soup.find_all('p')]

for p in soup.find_all('p'):
  print(p.text)

# print(text_lines)

Klassifiziere die Nachricht mit einem vortrainierten Modell mit der Huggingsface Bibliothek und PyTorch

In [31]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model_name = "nickmuchi/distilroberta-finetuned-financial-text-classification"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Dein Text
text = ""

for p in soup.find_all('p'):
  text = text + p.text + "\n"
# Tokenisiere den Text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Wende das Modell an
with torch.no_grad():
    outputs = model(**inputs)

# Hole die Modellvorhersagen
predictions = outputs.logits

# Konvertiere Vorhersagen in Wahrscheinlichkeiten und hole das maximale
predictions = torch.nn.functional.softmax(predictions, dim=1)
predicted_class = torch.argmax(predictions).item()

# Zuordnung der Klassen
classes = ["bearish", "neutral", "bullish"]

# Ergebnis
print(f"Sentiment: {classes[predicted_class]}")


Sentiment: bullish
